In [4]:
import os
import cv2
import numpy as np
from tqdm import tqdm

def split_and_resize(image_path, label_path, output_image_dir, output_label_dir, target_size=(1024, 1024), padding_threshold=0.3, max_crops=5, max_black_area=0.2):
    os.makedirs(output_image_dir, exist_ok=True)
    os.makedirs(output_label_dir, exist_ok=True)

    image = cv2.imread(image_path)
    h, w, _ = image.shape

    with open(label_path, 'r') as f:
        labels = [line.strip().split() for line in f.readlines()]

    # Convert YOLO normalized labels to pixel-based coordinates
    bbox_list = []
    for label in labels:
        class_id, cx, cy, bw, bh = map(float, label)
        cx, cy, bw, bh = cx * w, cy * h, bw * w, bh * h
        x1, y1 = cx - bw / 2, cy - bh / 2
        x2, y2 = cx + bw / 2, cy + bh / 2
        bbox_list.append((class_id, x1, y1, x2, y2))

    # Calculate stride dynamically to limit the number of crops
    max_stride_x = max(1, (w - target_size[0]) // max_crops)  # Max stride for width
    max_stride_y = max(1, (h - target_size[1]) // max_crops)  # Max stride for height
    stride_x = max(target_size[0] // 2, max_stride_x)
    stride_y = max(target_size[1] // 2, max_stride_y)

    output_count = 0

    for y in range(0, h, stride_y):
        for x in range(0, w, stride_x):
            cropped_image = image[y:y+target_size[1], x:x+target_size[0]]
            crop_h, crop_w, _ = cropped_image.shape

            # If the crop size is smaller than target, pad it
            if crop_h < target_size[1] or crop_w < target_size[0]:
                pad_top = 0
                pad_bottom = target_size[1] - crop_h
                pad_left = 0
                pad_right = target_size[0] - crop_w

                cropped_image = cv2.copyMakeBorder(cropped_image, pad_top, pad_bottom, pad_left, pad_right, cv2.BORDER_CONSTANT, value=(0, 0, 0))

            # Calculate the ratio of non-zero (non-black) area
            # We define black pixels as those with values below a certain threshold (e.g., 30 for RGB channels)
            black_pixels = np.sum(np.all(cropped_image < 30, axis=-1))  # Count pixels that are very dark
            black_area_ratio = black_pixels / (target_size[0] * target_size[1])

            if black_area_ratio > max_black_area:
                continue  # Skip crop if black area exceeds threshold

            # Calculate the ratio of non-zero (non-black) area
            non_zero_ratio = np.count_nonzero(cropped_image) / (target_size[0] * target_size[1] * 3)

            if non_zero_ratio < padding_threshold:
                continue  # Skip crop if non-zero area is below padding threshold

            # Adjust labels for the cropped area
            new_labels = []
            for class_id, x1, y1, x2, y2 in bbox_list:
                if x1 >= x+target_size[0] or x2 <= x or y1 >= y+target_size[1] or y2 <= y:
                    continue  # Skip boxes outside the crop

                nx1 = max(x1 - x, 0)
                ny1 = max(y1 - y, 0)
                nx2 = min(x2 - x, target_size[0])
                ny2 = min(y2 - y, target_size[1])

                # Recalculate normalized values
                ncx = (nx1 + nx2) / 2 / target_size[0]
                ncy = (ny1 + ny2) / 2 / target_size[1]
                nbw = (nx2 - nx1) / target_size[0]
                nbh = (ny2 - ny1) / target_size[1]

                new_labels.append(f"{int(class_id)} {ncx:.6f} {ncy:.6f} {nbw:.6f} {nbh:.6f}")

            if not new_labels:
                continue

            # Save the cropped image and label
            output_image_path = os.path.join(output_image_dir, f"{os.path.basename(image_path).split('.')[0]}_{output_count}.jpg")
            output_label_path = os.path.join(output_label_dir, f"{os.path.basename(label_path).split('.')[0]}_{output_count}.txt")

            cv2.imwrite(output_image_path, cropped_image)
            with open(output_label_path, 'w') as f:
                f.write("\n".join(new_labels))

            output_count += 1
            if output_count >= max_crops:
                return  # Stop once we reach the desired number of crops

# Example usage
input_image_dir = "C:/Users/Asus/Downloads/Computer_vision/Project/validation_dataset/images"
input_label_dir = "C:/Users/Asus/Downloads/Computer_vision/Project/validation_dataset/labels"
output_image_dir = "val_images"
output_label_dir = "val_labels"

image_files = [f for f in os.listdir(input_image_dir) if f.endswith(('.jpg', '.png'))]
label_files = [f.replace('.jpg', '.txt').replace('.png', '.txt') for f in image_files]

for image_file, label_file in tqdm(zip(image_files, label_files), total=len(image_files)):
    split_and_resize(
        os.path.join(input_image_dir, image_file),
        os.path.join(input_label_dir, label_file),
        output_image_dir,
        output_label_dir,
        max_crops=5 # Set the maximum number of crops per image
    )


100%|████████████████████████████████████████████████████████████████████████████████| 458/458 [01:47<00:00,  4.25it/s]


# Viewing the label on images

In [2]:
import os
import cv2
import numpy as np

image_folder = 'yolo_dataset/train/images'
label_folder = 'yolo_dataset/train/labels'
output_folder = 'labels_on_images'

os.makedirs(output_folder, exist_ok=True)

def draw_yolo_labels(image, label_file):
    with open(label_file, 'r') as f:
        labels = f.readlines()

    for label in labels:
        
        parts = label.strip().split()
        class_id = int(parts[0])
        x_center = float(parts[1])
        y_center = float(parts[2])
        width = float(parts[3])
        height = float(parts[4])

        
        img_height, img_width = image.shape[:2]

        
        x_center_pixel = int(x_center * img_width)
        y_center_pixel = int(y_center * img_height)
        width_pixel = int(width * img_width)
        height_pixel = int(height * img_height)

        
        top_left_x = int(x_center_pixel - width_pixel / 2)
        top_left_y = int(y_center_pixel - height_pixel / 2)
        bottom_right_x = int(x_center_pixel + width_pixel / 2)
        bottom_right_y = int(y_center_pixel + height_pixel / 2)

        
        color = (0, 255, 0) 
        thickness = 2  
        cv2.rectangle(image, (top_left_x, top_left_y), (bottom_right_x, bottom_right_y), color, thickness)

    return image

for image_name in os.listdir(image_folder):
    
    if image_name.lower().endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(image_folder, image_name)
        label_path = os.path.join(label_folder, image_name.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt'))

        if os.path.exists(label_path):
            
            image = cv2.imread(image_path)

            annotated_image = draw_yolo_labels(image, label_path)

            output_image_path = os.path.join(output_folder, image_name)
            cv2.imwrite(output_image_path, annotated_image)
            print(f"Processed and saved: {output_image_path}")
        else:
            print(f"Label file missing for image: {image_name}")


Processed and saved: labels_on_images\P0000_0.jpg
Processed and saved: labels_on_images\P0000_1.jpg
Processed and saved: labels_on_images\P0000_2.jpg
Processed and saved: labels_on_images\P0000_3.jpg
Processed and saved: labels_on_images\P0000_4.jpg
Processed and saved: labels_on_images\P0001_0.jpg
Processed and saved: labels_on_images\P0001_1.jpg
Processed and saved: labels_on_images\P0001_2.jpg
Processed and saved: labels_on_images\P0002_0.jpg
Processed and saved: labels_on_images\P0002_1.jpg
Processed and saved: labels_on_images\P0002_2.jpg
Processed and saved: labels_on_images\P0002_3.jpg
Processed and saved: labels_on_images\P0002_4.jpg
Processed and saved: labels_on_images\P0008_0.jpg
Processed and saved: labels_on_images\P0010_0.jpg
Processed and saved: labels_on_images\P0010_1.jpg
Processed and saved: labels_on_images\P0010_2.jpg
Processed and saved: labels_on_images\P0010_3.jpg
Processed and saved: labels_on_images\P0011_0.jpg
Processed and saved: labels_on_images\P0011_1.jpg


KeyboardInterrupt: 

# Below is to randomly distribute dataset

In [2]:
import os
import random
import shutil

# Paths to the image and label folders
image_folder = "train_images"
label_folder = "train_labels"

# Output folder structure
yolo_dataset = "yolo_dataset"
train_folder = os.path.join(yolo_dataset, "train")
val_folder = os.path.join(yolo_dataset, "val")

# Create the necessary directories
os.makedirs(train_folder, exist_ok=True)
os.makedirs(val_folder, exist_ok=True)
os.makedirs(os.path.join(train_folder, "images"), exist_ok=True)
os.makedirs(os.path.join(train_folder, "labels"), exist_ok=True)
os.makedirs(os.path.join(val_folder, "images"), exist_ok=True)
os.makedirs(os.path.join(val_folder, "labels"), exist_ok=True)

# Get list of image and label files
image_files = sorted(os.listdir(image_folder))
label_files = sorted(os.listdir(label_folder))

# Ensure there is a 1:1 correspondence between image and label files
assert len(image_files) == len(label_files), "Number of images and labels must match."

# Randomly shuffle the indices to split the data
indices = list(range(len(image_files)))
random.shuffle(indices)

# Define the split ratio (80% for training, 20% for validation)
split_ratio = 0.8
train_indices = indices[:int(len(indices) * split_ratio)]
val_indices = indices[int(len(indices) * split_ratio):]

# Function to copy files
def copy_files(indices, source_image_folder, source_label_folder, target_image_folder, target_label_folder):
    for idx in indices:
        # Get the corresponding image and label filenames
        image_file = image_files[idx]
        label_file = label_files[idx]
        
        # Construct the source and destination paths
        image_source = os.path.join(source_image_folder, image_file)
        label_source = os.path.join(source_label_folder, label_file)
        
        image_dest = os.path.join(target_image_folder, image_file)
        label_dest = os.path.join(target_label_folder, label_file)
        
        # Copy the files
        shutil.copy(image_source, image_dest)
        shutil.copy(label_source, label_dest)

# Copy the files to the "train" and "val" folders
copy_files(train_indices, image_folder, label_folder, os.path.join(train_folder, "images"), os.path.join(train_folder, "labels"))
copy_files(val_indices, image_folder, label_folder, os.path.join(val_folder, "images"), os.path.join(val_folder, "labels"))

print("Dataset has been split into 'train' and 'val' folders.")


Dataset has been split into 'train' and 'val' folders.


# Code to crop images only

In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm

def split_and_resize(image_path, output_image_dir, target_size=(1024, 1024), padding_threshold=0.3, max_crops=5, max_black_area=0.2):
    os.makedirs(output_image_dir, exist_ok=True)

    image = cv2.imread(image_path)
    h, w, _ = image.shape

    # Calculate stride dynamically to limit the number of crops
    max_stride_x = max(1, (w - target_size[0]) // max_crops)  # Max stride for width
    max_stride_y = max(1, (h - target_size[1]) // max_crops)  # Max stride for height
    stride_x = max(target_size[0] // 2, max_stride_x)
    stride_y = max(target_size[1] // 2, max_stride_y)

    output_count = 0

    for y in range(0, h, stride_y):
        for x in range(0, w, stride_x):
            cropped_image = image[y:y+target_size[1], x:x+target_size[0]]
            crop_h, crop_w, _ = cropped_image.shape

            # If the crop size is smaller than target, pad it
            if crop_h < target_size[1] or crop_w < target_size[0]:
                pad_top = 0
                pad_bottom = target_size[1] - crop_h
                pad_left = 0
                pad_right = target_size[0] - crop_w

                cropped_image = cv2.copyMakeBorder(cropped_image, pad_top, pad_bottom, pad_left, pad_right, cv2.BORDER_CONSTANT, value=(0, 0, 0))

            # Calculate the ratio of non-zero (non-black) area
            black_pixels = np.sum(np.all(cropped_image < 30, axis=-1))  # Count pixels that are very dark
            black_area_ratio = black_pixels / (target_size[0] * target_size[1])

            if black_area_ratio > max_black_area:
                continue  # Skip crop if black area exceeds threshold

            # Calculate the ratio of non-zero (non-black) area
            non_zero_ratio = np.count_nonzero(cropped_image) / (target_size[0] * target_size[1] * 3)

            if non_zero_ratio < padding_threshold:
                continue  # Skip crop if non-zero area is below padding threshold

            # Save the cropped image
            output_image_path = os.path.join(output_image_dir, f"{os.path.basename(image_path).split('.')[0]}_{output_count}.jpg")
            cv2.imwrite(output_image_path, cropped_image)

            output_count += 1
            if output_count >= max_crops:
                return  # Stop once we reach the desired number of crops

# Example usage
input_image_dir = "C:/Users/Asus/Downloads/Computer_vision/Project_2/test_images"
output_image_dir = "test_images_crop"

image_files = [f for f in os.listdir(input_image_dir) if f.endswith(('.jpg', '.png'))]

for image_file in tqdm(image_files, total=len(image_files)):
    split_and_resize(
        os.path.join(input_image_dir, image_file),
        output_image_dir,
        max_crops=5  # Set the maximum number of crops per image
    )


100%|████████████████████████████████████████████████████████████████████████████████| 469/469 [02:05<00:00,  3.75it/s]
